Reference: https://jupyterhub.readthedocs.io/en/stable/installation-guide.html

# core steps

### Create cluster using kops

In [3]:
%%bash
export NAME=jupyterhub.k8s.local

kops create cluster $NAME \
  --zones=us-west-2a,us-west-2b,us-west-2c \
  --authorization RBAC \
  --master-size t2.xlarge \
  --master-volume-size 20 \
  --node-size t2.xlarge \
  --node-volume-size 20 \
  --node-count 1 \
  --ssh-public-key $HOME/.ssh/id_rsa.jupyterhub.pub \
  --yes
sleep 360


*********************************************************************************

A new kubernetes version is available: 1.10.5
Upgrading is recommended (try kops upgrade cluster)

More information: https://github.com/kubernetes/kops/blob/master/permalinks/upgrade_k8s.md#1.10.5

*********************************************************************************

kops has set your kubectl context to jupyterhub.k8s.local

Cluster is starting.  It should be ready in a few minutes.

Suggestions:
 * validate cluster: kops validate cluster
 * list nodes: kubectl get nodes --show-labels
 * ssh to the master: ssh -i ~/.ssh/id_rsa admin@api.jupyterhub.k8s.local
 * the admin user is specific to Debian. If not using Debian please use the appropriate user based on your OS.
 * read about installing addons at: https://github.com/kubernetes/kops/blob/master/docs/addons.md.



I1003 21:27:14.341850   12769 create_cluster.go:1351] Using SSH public key: /Users/btsui/.ssh/id_rsa.jupyterhub.pub
I1003 21:27:15.417740   12769 create_cluster.go:480] Inferred --cloud=aws from zone "us-west-2a"
I1003 21:27:15.717691   12769 subnets.go:184] Assigned CIDR 172.20.32.0/19 to subnet us-west-2a
I1003 21:27:15.717724   12769 subnets.go:184] Assigned CIDR 172.20.64.0/19 to subnet us-west-2b
I1003 21:27:15.717734   12769 subnets.go:184] Assigned CIDR 172.20.96.0/19 to subnet us-west-2c
I1003 21:27:18.587214   12769 apply_cluster.go:505] Gossip DNS: skipping DNS validation
I1003 21:27:19.245704   12769 executor.go:103] Tasks: 0 done / 81 total; 30 can run
I1003 21:27:20.149052   12769 vfs_castore.go:735] Issuing new certificate: "ca"
I1003 21:27:20.267319   12769 vfs_castore.go:735] Issuing new certificate: "apiserver-aggregator-ca"
I1003 21:27:21.123302   12769 executor.go:103] Tasks: 30 done / 81 total; 26 can run
I1003 21:27:22.025680   12769 vfs_castore.go:735] Issuing new

### Install Tiller (Cluster version of pip install )

In [4]:
%%bash
kubectl --namespace kube-system create serviceaccount tiller
kubectl create clusterrolebinding tiller --clusterrole cluster-admin --serviceaccount=kube-system:tiller

serviceaccount "tiller" created
clusterrolebinding.rbac.authorization.k8s.io "tiller" created


In [5]:
!helm init --service-account tiller

$HELM_HOME has been configured at /Users/btsui/.helm.

Tiller (the Helm server-side component) has been installed into your Kubernetes Cluster.

Please note: by default, Tiller is deployed with an insecure 'allow unauthenticated users' policy.
To prevent this, run `helm init` with the --tiller-tls-verify flag.
For more information on securing your installation see: https://docs.helm.sh/using_helm/#securing-your-helm-installation
Happy Helming!


In [6]:
%%bash
!sleep 10
kubectl patch deployment tiller-deploy --namespace=kube-system --type=json --patch='[{"op": "add", "path": "/spec/template/spec/containers/0/command", "value": ["/tiller", "--listen=localhost:44134"]}]'

deployment.extensions "tiller-deploy" patched


### Set up EBS volumne

In [7]:
!kubectl apply -f storageclass.yml

storageclass.storage.k8s.io "gp2" configured


In [8]:
!sleep 60

Set up the DNS: 

https://console.aws.amazon.com/route53/home

Choose Go to Record Sets.
Choose Create Record Set.
For Create Record Set, do the following:
Leave the default name, which is the name of your domain.
For Type, select A — IPv4 address.
For Alias, choose Yes. An alias enables Route 53 to associate your domain name with an AWS resource, such as a load balancer.
Choose Alias Target. Select your load balancer from the list. The console adds the dualstack prefix.
For Routing Policy, select Simple.
Leave Evaluate Target Health set to No.


# Extra

### Set up EFS

##### delete existing mount targets on EFS

In [13]:
import boto3
import pandas as pd

efs = boto3.client('efs')

efsId='fs-e1636448'

In [14]:
for Dict in efs.describe_mount_targets(FileSystemId=efsId)['MountTargets']:
    efs.delete_mount_target(MountTargetId=Dict['MountTargetId'])

### mount targets

In [15]:
ec2 = boto3.resource('ec2')

instances=list(ec2.instances.all())

newVPC=pd.Series(list(map(lambda i:i.vpc_id,instances))).dropna().iloc[0]
print ('new VPC ID:',newVPC)

new VPC ID: vpc-020d6c6d174e3674c


In [16]:
newSubnets=pd.Series(list(map(lambda i:i.subnet_id,instances))).dropna()#.iloc[0]

In [17]:
groupIdL=[]
for ec2 in instances:
    if len(ec2.security_groups)>0:
        groupIdL.append((ec2.security_groups[0]['GroupId'])) 
#newSgs=pd.Series(list(map(lambda i:i.security_groups[0]['GroupId'],instances))).dropna()#.iloc[0]

In [18]:
efsId,newSubnets

('fs-e1636448', 1    subnet-03b6b36296cf45cb7
 4    subnet-04aab7a6afc9f418a
 dtype: object)

In [19]:
newSubnets

1    subnet-03b6b36296cf45cb7
4    subnet-04aab7a6afc9f418a
dtype: object

In [20]:
groupIdL

['sg-0edb0763f38225af9', 'sg-048d8df9c4c910075']

In [21]:
#newVPC=''
for SubnetId in newSubnets:
    efs.create_mount_target(FileSystemId=efsId,
        SubnetId=SubnetId,SecurityGroups=groupIdL
    )

In [ ]:
#!wget -O manifest.helm.yaml https://raw.githubusercontent.com/kubernetes-incubator/external-storage/master/aws/efs/deploy/manifest.yaml
asdasdasd

### mount NFS

In [24]:
!sleep 200

In [25]:
import boto3
import botocore
import paramiko

key = paramiko.RSAKey.from_private_key_file('/Users/btsui/.ssh/id_rsa.jupyterhub')
client = paramiko.SSHClient()
client.set_missing_host_key_policy(paramiko.AutoAddPolicy())

In [26]:
import boto3

ec2 = boto3.resource('ec2')

instances=list(ec2.instances.all())

ec2=instances[0
             ]

for ec2 in instances:
    if ec2.public_dns_name !='' :
        # Connect/ssh to an instance
        instance_ip=ec2.public_dns_name
        #try:
        # Here 'ubuntu' is user name and 'instance_ip' is public IP of EC2
        client.connect(hostname=instance_ip, username="admin", pkey=key)
        # Execute a command(cmd) after connecting/ssh to an instance
        cmd='mkdir ~/efs'
        stdin, stdout, stderr = client.exec_command(cmd)
        cmd='sudo mount -t nfs4 -o nfsvers=4.1,rsize=1048576,wsize=1048576,hard,timeo=600,retrans=2 fs-e1636448.efs.us-west-2.amazonaws.com:/ ~/efs'
        stdin, stdout, stderr = client.exec_command(cmd)
        print (stdout.read())
        print (stderr.read())
        client.close()

b''
b''
b''
b''


### setup jupyter hub

In [27]:
%%bash
RELEASE=jhub
NAMESPACE=jhub
helm upgrade --install $RELEASE jupyterhub/jupyterhub \
  --namespace $NAMESPACE  \
  --version 0.7.0 \
  --values config.yaml

Release "jhub" does not exist. Installing it now.
NAME:   jhub
LAST DEPLOYED: Wed Oct  3 21:42:01 2018
NAMESPACE: jhub
STATUS: DEPLOYED

RESOURCES:
==> v1beta1/PodDisruptionBudget
NAME   MIN AVAILABLE  MAX UNAVAILABLE  ALLOWED DISRUPTIONS  AGE
hub    1              N/A              0                    0s
proxy  1              N/A              0                    0s

==> v1/Secret
NAME        TYPE    DATA  AGE
hub-secret  Opaque  1     0s

==> v1/PersistentVolumeClaim
NAME        STATUS   VOLUME  CAPACITY  ACCESS MODES  STORAGECLASS  AGE
hub-db-dir  Pending  gp2     0s

==> v1/ServiceAccount
NAME  SECRETS  AGE
hub   1        0s

==> v1/Service
NAME          TYPE          CLUSTER-IP     EXTERNAL-IP  PORT(S)       AGE
hub           ClusterIP     100.71.188.34  <none>       8081/TCP      0s
proxy-api     ClusterIP     100.67.69.165  <none>       8001/TCP      0s
proxy-public  LoadBalancer  100.70.67.13   <pending>    80:31469/TCP  0s

==> v1beta2/Deployment
NAME   DESIRED  CURRENT  UP-TO

E1003 21:44:31.829819   12933 portforward.go:303] error copying from remote stream to local connection: readfrom tcp4 127.0.0.1:56619->127.0.0.1:56623: write tcp4 127.0.0.1:56619->127.0.0.1:56623: write: broken pipe


### Connect DNS with load balancer IP
* [DNS](https://console.aws.amazon.com/route53/home?region=us-west-2#resource-record-sets:Z3ALGFGOLRJSAE)



In [3]:
"""
import boto3

client53 = boto3.client('route53')
response = client53.change_resource_record_sets(
    HostedZoneId='Z1H1FL5HABSF5',
    ChangeBatch={
        'Comment': 'string',
        'Changes': [
            {
                'Action': 'UPSERT',
                'ResourceRecordSet': {
                    'Name': 'jupyterhub.hannahcarterlab.org.',
                    'Type': 'A',
                    #'SetIdentifier': 'string',
                    #'Weight': 123,
                    'Region': 'us-west-2',#'us-east-1'|'us-east-2'|'us-west-1'|'us-west-2'|'ca-central-1'|'eu-west-1'|'eu-west-2'|'eu-west-3'|'eu-central-1'|'ap-southeast-1'|'ap-southeast-2'|'ap-northeast-1'|'ap-northeast-2'|'ap-northeast-3'|'sa-east-1'|'cn-north-1'|'cn-northwest-1'|'ap-south-1',
                    #'Failover': 'PRIMARY'|'SECONDARY',
                    #'MultiValueAnswer': True|False,
                    #'TTL': 123,
                    'AliasTarget': {
                        'HostedZoneId': 'dualstack.a30f630e7c73411e8aec902ea44e5fc5-311761146.us-west-2.elb.amazonaws.com.',
                        'DNSName': 'jupyterhub.hannahcarterlab.org',
                        'EvaluateTargetHealth': False
                    },
                    #'HealthCheckId': 'string',
                    #'TrafficPolicyInstanceId': 'string'
                }
            },
        ]
    }
)
"""

"\nimport boto3\n\nclient53 = boto3.client('route53')\nresponse = client53.change_resource_record_sets(\n    HostedZoneId='Z1H1FL5HABSF5',\n    ChangeBatch={\n        'Comment': 'string',\n        'Changes': [\n            {\n                'Action': 'UPSERT',\n                'ResourceRecordSet': {\n                    'Name': 'jupyterhub.hannahcarterlab.org.',\n                    'Type': 'A',\n                    #'SetIdentifier': 'string',\n                    #'Weight': 123,\n                    'Region': 'us-west-2',#'us-east-1'|'us-east-2'|'us-west-1'|'us-west-2'|'ca-central-1'|'eu-west-1'|'eu-west-2'|'eu-west-3'|'eu-central-1'|'ap-southeast-1'|'ap-southeast-2'|'ap-northeast-1'|'ap-northeast-2'|'ap-northeast-3'|'sa-east-1'|'cn-north-1'|'cn-northwest-1'|'ap-south-1',\n                    #'Failover': 'PRIMARY'|'SECONDARY',\n                    #'MultiValueAnswer': True|False,\n                    #'TTL': 123,\n                    'AliasTarget': {\n                        'Hosted

# Useful commands

### update the server

In [29]:
%%bash
RELEASE=jhub

helm upgrade $RELEASE jupyterhub/jupyterhub \
  --version=0.7.0 \
  --values config.yaml

Release "jhub" has been upgraded. Happy Helming!
LAST DEPLOYED: Wed Oct  3 21:45:28 2018
NAMESPACE: jhub
STATUS: DEPLOYED

RESOURCES:
==> v1beta2/Deployment
NAME   DESIRED  CURRENT  UP-TO-DATE  AVAILABLE  AGE
hub    1        1        1           1          1m
proxy  1        1        1           1          1m

==> v1beta1/PodDisruptionBudget
NAME   MIN AVAILABLE  MAX UNAVAILABLE  ALLOWED DISRUPTIONS  AGE
hub    1              N/A              0                    1m
proxy  1              N/A              0                    1m

==> v1/Pod(related)
NAME                    READY  STATUS   RESTARTS  AGE
hub-5c789b49f7-pcb6x    1/1    Running  0         1m
proxy-64c555c54c-cwhrl  1/1    Running  0         1m

==> v1/Secret
NAME        TYPE    DATA  AGE
hub-secret  Opaque  1     1m

==> v1/ConfigMap
NAME        DATA  AGE
hub-config  40    1m

==> v1/PersistentVolumeClaim
NAME        STATUS  VOLUME                                    CAPACITY  ACCESS MODES  STORAGECLASS  AGE
hub-db-dir  Boun

In [ ]:
! kubectl --namespace=jhub get pod

In [ ]:
!cat config.yaml

### Build the docker image

In [ ]:
%%bash
cd /Users/btsui/GithubProject/CarterLabJupyterHub
docker build -t jhubuserimage .
docker tag jhubuserimage btsui/get-started:v1
docker push btsui/get-started:v1

my docker file

In [10]:
!cat /Users/btsui/GithubProject/CarterLabJupyterHub/Docker/Dockerfile

FROM jupyter/minimal-notebook:177037d09156
RUN pip install nbgitpuller
RUN jupyter serverextension enable --py nbgitpuller --sys-prefix
RUN conda create --yes -n skymap jupyter python=3.6 pandas=0.23.4 seaborn=0.8.1 scikit-learn=0.19.1
RUN conda install --yes -n skymap nb_conda_kernels
RUN conda install --yes nb_conda_kernels


#RUN conda install --yes astropy


## AWS Console short cut

* [EC2](https://us-west-2.console.aws.amazon.com/ec2/v2/home?region=us-west-2#Instances:sort=availabilityZone)
* [DNS](https://console.aws.amazon.com/route53/home?region=us-west-2#)
* [EFS](https://us-west-2.console.aws.amazon.com/efs/home?region=us-west-2#/filesystems)


### delete the cluster

[Delete mount targets first](#delete-existing-mount-targets-on-EFS)

In [ ]:
#! kops delete cluster jupyterhub.k8s.local --yes

# Create Kubernete dashboard

[start kube server](startKubServer.ipynb)



In [ ]:
### Set up dashboard

!kubectl apply -f https://raw.githubusercontent.com/kubernetes/dashboard/master/src/deploy/recommended/kubernetes-dashboard.yaml

!kubectl apply -f admin.yml

!kubectl apply -f admin_previledge.yml

!kubectl -n kube-system describe secret $(kubectl -n kube-system get secret | grep admin-user | awk '{print $1}')

http://localhost:8001/api/v1/namespaces/kube-system/services/https:kubernetes-dashboard:/proxy/



### Find load balancer IP name



In [ ]:


%%bash
kubectl describe service proxy-public  --namespace jhub

In [2]:
!kubectl get nodes

NAME                                         STATUS    ROLES     AGE       VERSION
ip-172-20-39-64.us-west-2.compute.internal   Ready     master    1d        v1.10.3
ip-172-20-95-93.us-west-2.compute.internal   Ready     node      1d        v1.10.3
